## Testing Polynomials
We'll start with our imports as well as importing the rationals.

In [1]:
from groebner.polynomials import PolynomialRing, Polynomial

We can either instantiate `PolynomialRing` using the `num_vars` parameter or the `labels` parameter. If you provide labels, the `num_vars` parameter will be ignored.

In [2]:
# instantiating with custom labels
R = PolynomialRing(labels=['x','y','z'], base_field='QQ')
x, y, z = R.get_vars()

In [3]:
f = 3/4*x + 2*y

/home/nico/documents/math/groebner/groebner/rationals.py:25: RuntimeWarning: Coercing to Rational from float is inherently inexact and will be incorrect for large numerators and denominators. If you need to have certainty the values are correct, you should directly instantiate with Rational(num, den) with integers.
  warn(


In [4]:
f*z**2

3/4xz^2 + 2yz^2

In [5]:
f**10

59049/1048576x^10 + 98415/65536x^9y + 295245/16384x^8y^2 + 32805/256x^7y^3 + 76545/128x^6y^4 + 15309/8x^5y^5 + 8505/2x^4y^6 + 6480x^3y^7 + 6480x^2y^8 + 3840xy^9 + 1024y^10

In [6]:
# coercion works
(x + y)*(x - y) + 1

x^2 - y^2 + 1

Using the other form of instantiation is more convenient because we don't have to bother writing out all the labels, but it can lead to some confusing results as below. Of course all of this can happen with bad variable naming.

In [7]:
S = PolynomialRing(num_vars=3, base_field='QQ')
x3, x2, x1 = S.get_vars()

In [8]:
x1

x_2

In [9]:
x1-x3**3

-x_0^3 + x_2

### Membership and types
These mostly work as you would hope.

In [10]:
x in R

True

In [11]:
2*x+3/4*y in R

True

In [12]:
x in R.field

False

In [13]:
type(2*x+3/4*y) is Polynomial

True

### Monomials and orderings
The core tool here is the `MonomialOrdering` class that gives us a way to well-order all monomials. Our implementation of `Polynomial` is basically just a list of coefficients and the ordering handles conversion between list index and monomial.

In [14]:
from groebner.monomials import MonomialOrdering, Monomial

In [15]:
# graded lexicographic ordering is the only one I have implemented.
o = MonomialOrdering(num_vars=3, labels=['x','y','w'], order_type='grlex')

In [16]:
for i in range(4, 21):
    print(f'The {i}th monomial is {o.idx_to_monomial(i)}')

The 4th monomial is w^2
The 5th monomial is yw
The 6th monomial is y^2
The 7th monomial is xw
The 8th monomial is xy
The 9th monomial is x^2
The 10th monomial is w^3
The 11th monomial is yw^2
The 12th monomial is y^2w
The 13th monomial is y^3
The 14th monomial is xw^2
The 15th monomial is xyw
The 16th monomial is xy^2
The 17th monomial is x^2w
The 18th monomial is x^2y
The 19th monomial is x^3
The 20th monomial is w^4


In [17]:
# quick check that when we convert back to an index we get the same thing
# if you move the range up much higher be prepared to wait...
for i in range(10000):
    m = o.idx_to_monomial(i)
    assert i == m.to_idx()

Some examination of the insides of a monomial

In [18]:
# 12 is the "index", or order in which it appears in the list of all monomials
m = o.idx_to_monomial(12)
print(m)

y^2w


In [19]:
m.to_idx()

12

In [20]:
m.degrees

[0, 2, 1]

In [21]:
m.total_degree

3

This is the level on which multiplication is implemented, so we can multiply two monomials (to get another monomial)

In [22]:
m*m

y^4w^2

In [23]:
type(m*m) is Monomial

True

We also have comparison (using the ordering given by the ordering `o`)

In [24]:
s = Monomial([1,2,2], o)
t = Monomial([2,1,1], o)
print(s," || ",t)

xy^2w^2  ||  x^2yw


In [25]:
s > t

True